In [2]:
import matplotlib.pyplot as plt 
import numpy as np 
import sympy as sp 
from scipy.optimize import brentq

In [20]:
t,a,b = sp.symbols('t,a,b')

cardiod = sp.Matrix([2*(1-sp.cos(t))*sp.cos(t),2*(1-sp.cos(t))*sp.sin(t),0])
circle = sp.Matrix([sp.sin(t),sp.cos(t),sp.sin(t)])
torus = sp.Matrix([sp.cos(t)*(4+2*sp.cos(5*t)),sp.sin(t)*(4+2*sp.cos(5*t)),2*sp.cos(5*t)])
xminus2 = sp.Matrix([t,t**-2,t])
lemniscate = sp.Matrix([sp.sin(t),sp.sin(2*t),0])
funnel = sp.Matrix([sp.sin(t**2)/t,sp.cos(t**2)/t,t])
logarithmic_curve = sp.Matrix([sp.cos(10*t)*sp.exp(t),sp.sin(10*t)*sp.exp(t),0])
asteroid = sp.Matrix([sp.cos(t)**3,sp.sin(t)**3,0])
idk = sp.Matrix([t,t**2,t**4])
gamma = funnel

gamma_prime = sp.diff(gamma,t)
gamma_prime_a_dot_b = gamma_prime.subs(t,a).T @ gamma_prime.subs(t,b)
gamma_prime_a_dot_b_lambda = sp.lambdify((a,b),gamma_prime_a_dot_b[0])
gamma_prime_lambda = sp.lambdify(t,gamma_prime)
gamma_lambda = sp.lambdify(t,gamma)


gamma_domain = [-6,6]

Bs = []
orthoptic = []
for a in np.linspace(gamma_domain[0],gamma_domain[1],1000):
    gamma_prime_a_dot_b = lambda b: gamma_prime_a_dot_b_lambda(a,b)
    b_values = np.linspace(gamma_domain[0],gamma_domain[1],10000)
    dot_product_values = gamma_prime_a_dot_b(b_values)

    sign_changes = np.where(np.diff(np.sign(dot_product_values)))[0]
    B = []

    for idx in sign_changes:
        b_left = b_values[idx]
        b_right = b_values[idx+1]
        try:
            root = brentq(gamma_prime_a_dot_b, b_left, b_right)
            B.append(root)
        except ValueError:
            # In case the interval doesn't bracket a root properly
            pass
    Bs.append(B)

    for b in B:
        try:
            damatrix = np.array([gamma_prime_lambda(a).squeeze(),-gamma_prime_lambda(b).squeeze()]).T
            [t,s], residuals, rank, s = np.linalg.lstsq(damatrix, (gamma_lambda(b)-gamma_lambda(a)).flatten(),rcond=None)
            if rank == 2:
                orthoptic.append(gamma_prime_lambda(a)*t + gamma_lambda(a))
        except ZeroDivisionError:
            pass

In [22]:
orthoptic = np.array(orthoptic).squeeze(-1)

ValueError: cannot select an axis to squeeze out which has size not equal to one

In [24]:
%matplotlib tk
Xs = orthoptic[:,0]
Ys = orthoptic[:,1]
gamma = gamma_lambda(np.linspace(gamma_domain[0],gamma_domain[1],1000)).squeeze(-2)
plt.plot(gamma[0],gamma[1])
plt.scatter(Xs,Ys,s=.3,c='red')
plt.xlim(-5,5)
plt.ylim(-5,5)

(-5.0, 5.0)

In [25]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # Import the 3D plotting toolkit

# Extract coordinates from orthoptic (assumed to be an Nx3 array)
Xs = orthoptic[:, 0]
Ys = orthoptic[:, 1]
Zs = orthoptic[:, 2]

# Compute the 3D curve gamma (assumed gamma_lambda returns an array with three components)
# Here we generate 1000 points in the parameter domain.
gamma = np.zeros((1000,3))
for i in range(1000):
    t = gamma_domain[1]*20*((i/1000)-0.5)
    gamma[i] = gamma_lambda(t).flatten()


# Create a figure and a 3D axis
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot the 3D curve
ax.plot(gamma[:,0], gamma[:,1], gamma[:,2], label='3D curve')

# Scatter plot the orthoptic points in red
ax.scatter(Xs, Ys, Zs, s=0.3, c='red', label='Orthoptic points')

# Set axis limits
ax.set_xlim([-5, 5])
ax.set_ylim([-5, 5])
ax.set_zlim([-5, 5])

ax.legend()
plt.show()


<lambdifygenerated-12>:2: RuntimeWarning: invalid value encountered in scalar divide
  return array([[sin(t**2)/t], [cos(t**2)/t], [t]])
<lambdifygenerated-12>:2: RuntimeWarning: divide by zero encountered in scalar divide
  return array([[sin(t**2)/t], [cos(t**2)/t], [t]])


In [27]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # required for 3D plotting
from matplotlib.animation import FuncAnimation
%matplotlib tk
As = np.linspace(gamma_domain[0],gamma_domain[1],1000)
p = 2
# Set up the figure and a 3D axes
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Extract coordinates from orthoptic (assumed to be an Nx3 array)
Xs = orthoptic[:, 0]
Ys = orthoptic[:, 1]
Zs = orthoptic[:, 2]

# Compute the 3D curve gamma (assumed gamma_lambda returns an array with three components)
# Here we generate 1000 points in the parameter domain.
gamma = np.zeros((1000,3))
for i in range(1000):
    try:
        t = gamma_domain[1]*2*((i/1000)-0.5)
        gamma[i] = gamma_lambda(t).flatten()
    except ZeroDivisionError:
        pass

# Plot the 3D curve
ax.plot(gamma[:,0], gamma[:,1], gamma[:,2], label='3D curve')

# Scatter plot the orthoptic points in red
ax.scatter(Xs, Ys, Zs, s=0.3, c='red', label='Orthoptic points')

# Set axis limits
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
# Set axis limits
ax.set_xlim([-5, 5])
ax.set_ylim([-5, 5])
ax.set_zlim([-5, 5])
# Animation update function
def animate(i):
    a = As[i*p]
    bs = Bs[i*p]
    r1 = gamma_lambda(a)-gamma_prime_lambda(a)*100/np.linalg.norm(gamma_prime_lambda(a))
    r2 = gamma_lambda(a)+gamma_prime_lambda(a)*100/np.linalg.norm(gamma_prime_lambda(a))
    tangent, = ax.plot([r1[0],r2[0]], [r1[1],r2[1]], [r1[2],r2[2]], lw=2, color='red')
    perp_tangents = []
    for b in bs:

        r1 = gamma_lambda(b)-gamma_prime_lambda(b)*100/np.linalg.norm(gamma_prime_lambda(b))
        r2 = gamma_lambda(b)+gamma_prime_lambda(b)*100/np.linalg.norm(gamma_prime_lambda(b))
        perp_tangents.append(*ax.plot([r1[0],r2[0]], [r1[1],r2[1]], [r1[2],r2[2]], lw=2, color='blue'))
    print(perp_tangents)
    return [tangent]+perp_tangents

# Create the animation object
ani = FuncAnimation(fig, animate, frames=len(As)//p,blit=True,
                    interval=50)

plt.show()


[<mpl_toolkits.mplot3d.art3d.Line3D object at 0x00000228846472D0>, <mpl_toolkits.mplot3d.art3d.Line3D object at 0x00000228873F6D10>, <mpl_toolkits.mplot3d.art3d.Line3D object at 0x00000228873F75D0>, <mpl_toolkits.mplot3d.art3d.Line3D object at 0x00000228873F7DD0>, <mpl_toolkits.mplot3d.art3d.Line3D object at 0x000002288735C650>, <mpl_toolkits.mplot3d.art3d.Line3D object at 0x000002288735D010>, <mpl_toolkits.mplot3d.art3d.Line3D object at 0x000002288735D850>, <mpl_toolkits.mplot3d.art3d.Line3D object at 0x000002288735DFD0>, <mpl_toolkits.mplot3d.art3d.Line3D object at 0x000002288735E810>, <mpl_toolkits.mplot3d.art3d.Line3D object at 0x000002288735EFD0>, <mpl_toolkits.mplot3d.art3d.Line3D object at 0x000002288735F750>, <mpl_toolkits.mplot3d.art3d.Line3D object at 0x000002288735FF90>, <mpl_toolkits.mplot3d.art3d.Line3D object at 0x000002288733C790>, <mpl_toolkits.mplot3d.art3d.Line3D object at 0x000002288733D010>, <mpl_toolkits.mplot3d.art3d.Line3D object at 0x000002288733D690>, <mpl_tool

In [28]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # required for 3D plotting
from matplotlib.animation import FuncAnimation
%matplotlib tk
As = np.linspace(gamma_domain[0],gamma_domain[1],10000)
p = 20
# Set up the figure and a 3D axes
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Extract coordinates from orthoptic (assumed to be an Nx3 array)
Xs = orthoptic[:, 0]
Ys = orthoptic[:, 1]
Zs = orthoptic[:, 2]

# Compute the 3D curve gamma (assumed gamma_lambda returns an array with three components)
# Here we generate 1000 points in the parameter domain.
gamma = np.zeros((1000,3))
for i in range(1000):
    try:
        t = gamma_domain[1]*2*((i/1000)-0.5)
        gamma[i] = gamma_lambda(t).flatten()
    except ZeroDivisionError:
        pass

# Plot the 3D curve
ax.plot(gamma[:,0], gamma[:,1], gamma[:,2], label='3D curve')


# Set axis limits
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
# Set axis limits
ax.set_xlim([-5, 5])
ax.set_ylim([-5, 5])
ax.set_zlim([-5, 5])
# Animation update function
def animate(i):
    a = As[i*p]
    bs = Bs[i*p]
    r1 = gamma_lambda(a)-gamma_prime_lambda(a)*10
    r2 = gamma_lambda(a)+gamma_prime_lambda(a)*10
    tangent, = ax.plot([r1[0],r2[0]], [r1[1],r2[1]], [r1[2],r2[2]], lw=2, color='red')
    perp_tangents = []
    for b in bs:

        r1 = gamma_lambda(b)-gamma_prime_lambda(b)*10
        r2 = gamma_lambda(b)+gamma_prime_lambda(b)*10
        perp_tangents.append(*ax.plot([r1[0],r2[0]], [r1[1],r2[1]], [r1[2],r2[2]], lw=2, color='blue'))
    
    
    # Scatter plot the orthoptic points in red
    scatter = ax.scatter(Xs[:i*p], Ys[:i*p], Zs[:i*p], s=0.3, c='red')
    return [tangent]+perp_tangents

# Create the animation object
ani = FuncAnimation(fig, animate, frames=len(As)//p,blit=True,
                    interval=50)

plt.show()
